In [39]:
from statsbombpy import sb

import pandas as pd

import json

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
#from sklearn.preprocessing import TargetEncoder

import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
competitions = sb.competitions()

In [4]:
#competitions

In [5]:
#competitions['competition_name'].unique()

In [6]:
competitions[
    (competitions['competition_name'].isin([
    'Premier League', '1. Bundesliga', 'La Liga', 'Ligue 1', 'Serie A']))
             & 
    (competitions['season_name']=='2015/2016')
            ]

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-12-12T07:43:33.436182,None,None,2023-12-12T07:43:33.436182
41,11,27,Spain,La Liga,male,False,False,2015/2016,2023-09-20T17:21:32.111535,2021-06-13T16:17:31.694,None,2023-09-20T17:21:32.111535
58,7,27,France,Ligue 1,male,False,False,2015/2016,2023-12-13T00:27:57.162379,None,None,2023-12-13T00:27:57.162379
62,2,27,England,Premier League,male,False,False,2015/2016,2023-11-14T11:25:33.476498,2021-06-13T16:17:31.694,None,2023-11-14T11:25:33.476498
64,12,27,Italy,Serie A,male,False,False,2015/2016,2023-12-13T17:32:46.423081,None,None,2023-12-13T17:32:46.423081


In [7]:
#competitions[competitions['competition_name']=='Champions League']

In [8]:
PremierLeague = sb.matches(competition_id=2, season_id=27)
#PremierLeague

In [9]:
relevant_attributes = ['minute', 'team', 'shot_statsbomb_xg', 'location', 'shot_body_part',
       'shot_end_location', 'shot_first_time', 
         'shot_technique',
       'shot_type', 'type','shot_outcome']

league_df = pd.DataFrame(columns=relevant_attributes)
start = time.time()
for match in PremierLeague['match_id'][:10]:
    match_df = sb.events(match_id=match)
    #relevant_match_df = match_df[relevant_attributes].dropna(subset=['shot_statsbomb_xg'])
    relevant_match_df = match_df[relevant_attributes][match_df['type']=='Shot']
    #relevant_match_df = match_df[match_df['type']=='Shot']
    league_df = pd.concat([league_df, relevant_match_df], ignore_index=True)
# league_df
#
#print(PremierLeague['match_id'][0:10])
end = time.time()
print(f'script run time: {(end - start)/60} minutes')
league_df['shot_outcome'] = league_df['shot_outcome'].replace(to_replace={'Off T': 0, 'Blocked': 0, 'Saved': 0, 'Goal': 1, 'Wayward': 0, 'Post': 0,
               'Saved to Post': 0, 'Saved Off Target': 0})

league_df['distance'] = league_df.apply(lambda row: np.sqrt(np.abs(row.location[0]-120)**2 + 
                                                     np.abs(row.location[1]-40)**2), axis=1)
league_df

script run time: 0.18485538164774576 minutes


,minute,team,shot_statsbomb_xg,location,shot_body_part,shot_end_location,shot_first_time,shot_technique,shot_type,type,shot_outcome,distance
0,1,AFC Bournemouth,0.020838,"[107.1, 26.6]",Right Foot,"[109.7, 30.1]",NaN,Normal,Open Play,Shot,0,18.600269
1,4,AFC Bournemouth,0.034462,"[113.7, 54.0]",Right Foot,"[114.5, 52.6]",NaN,Normal,Open Play,Shot,0,15.352199
2,9,Leicester City,0.035481,"[115.7, 50.5]",Left Foot,"[120.0, 43.3, 3.6]",True,Overhead Kick,Open Play,Shot,0,11.346365
3,13,AFC Bournemouth,0.383711,"[113.9, 38.6]",Head,"[120.0, 46.0, 1.2]",NaN,Normal,Open Play,Shot,0,6.258594
4,25,Leicester City,0.036740,"[96.7, 47.2]",Right Foot,"[103.9, 40.1]",NaN,Half Volley,Open Play,Shot,0,24.387087
...,...,...,...,...,...,...,...,...,...,...,...,...
236,81,Crystal Palace,0.400244,"[113.5, 40.8]",Right Foot,"[113.6, 42.3]",True,Volley,Open Play,Shot,0,6.549046
237,81,Crystal Palace,0.098340,"[111.7, 40.0]",Head,"[120.0, 44.1, 4.5]",NaN,Normal,Open Play,Shot,0,8.300000
238,83,Everton,0.022453,"[89.8, 42.8]",Left Foot,"[117.4, 39.6, 2.0]",NaN,Normal,Open Play,Shot,0,30.329524
239,86,Crystal Palace,0.448667,"[115.9, 37.0]",Head,"[120.0, 34.7, 3.5]",NaN,Normal,Open Play,Shot,0,5.080354


In [10]:
league_df['shot_outcome'].value_counts()

shot_outcome
0    224
1     17
Name: count, dtype: int64

In [11]:
sum(league_df['shot_statsbomb_xg'])

25.524680622199995

In [13]:
cap_x_df, y_df = pd.concat([league_df.iloc[:,:-2], league_df.iloc[:,-1]], axis=1), league_df.iloc[:,-2].to_frame()

In [14]:
train_cap_x_df, test_cap_x_df, train_y_df, test_y_df = \
train_test_split(cap_x_df, y_df,
                 test_size=0.20,
                random_state=42,
                 shuffle=True,
                stratify=y_df)

In [15]:
target_attr = 'shot_outcome'

In [16]:
numerical_attrs = ['distance']

In [17]:
nominal_attrs = ['shot_technique', 'shot_type', 'shot_body_part']

In [18]:
numerical_transformer = Pipeline(
        steps=[("imputer", SimpleImputer()),
               ("scaler", StandardScaler())]
    )

In [27]:
nominal_transformer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="most_frequent")),
               #('target_encoder', TargetEncoder(target_type=target_type, random_state=42)),
               ("target_encoder", OrdinalEncoder()),
               ("scaler", StandardScaler())
               ]
    )

In [28]:
preprocessor = ColumnTransformer(
        transformers=[
            ('numerical', numerical_transformer, numerical_attrs),
            ('nominal', nominal_transformer, nominal_attrs)
        ]
    )

In [29]:
estimator = LogisticRegression()

In [30]:
composite_estimator = Pipeline(steps=[('preprocessor', preprocessor), ('estimator', estimator)])

In [31]:
composite_estimator.fit(train_cap_x_df, train_y_df.values.ravel())

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['distance']),
                                                 ('nominal',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('target_encoder',
                                                                   OrdinalEncoder()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['shot_technique',
                                                   'shot_type',
                                                   'shot_body_part'])])),
                ('estimator', LogisticRegression())])

In [32]:
composite_estimator.predict_proba(train_cap_x_df)

array([[7.96131316e-01, 2.03868684e-01],
       [9.89316049e-01, 1.06839506e-02],
       [8.73689887e-01, 1.26310113e-01],
       [9.88419243e-01, 1.15807568e-02],
       [9.95141779e-01, 4.85822130e-03],
       [9.95557709e-01, 4.44229106e-03],
       [9.27908608e-01, 7.20913919e-02],
       [9.03023729e-01, 9.69762711e-02],
       [9.91300676e-01, 8.69932413e-03],
       [9.16150568e-01, 8.38494320e-02],
       [9.69562389e-01, 3.04376111e-02],
       [9.95705244e-01, 4.29475591e-03],
       [9.76064832e-01, 2.39351677e-02],
       [9.52779496e-01, 4.72205042e-02],
       [9.21198802e-01, 7.88011985e-02],
       [9.52120285e-01, 4.78797154e-02],
       [9.99658473e-01, 3.41526792e-04],
       [9.33285833e-01, 6.67141668e-02],
       [9.78377733e-01, 2.16222672e-02],
       [8.89937053e-01, 1.10062947e-01],
       [9.37873384e-01, 6.21266157e-02],
       [9.85162368e-01, 1.48376320e-02],
       [9.28993124e-01, 7.10068758e-02],
       [9.00988908e-01, 9.90110922e-02],
       [9.622405

In [51]:
print("Training set actual goal information")
print(train_y_df[target_attr].value_counts())
print("Training set naive model total xG",sum(composite_estimator.predict_proba(train_cap_x_df)[:,1]))
print("Training set Statsbomb total xG:", sum(train_cap_x_df['shot_statsbomb_xg']))

Training set actual goal information
shot_outcome
0    178
1     14
Name: count, dtype: int64
Training set naive model total xG 14.00000157795762
Training set Statsbomb total xG: 21.929878123800002


Actual goal information
shot_outcome
0    178
1     14
Name: count, dtype: int64


21.929878123800002

In [ ]:
classification_threshold = 0.5

In [41]:
test_predictions = composite_estimator.predict_proba(test_cap_x_df)

In [50]:
print("Test set actual goal information")
print(test_y_df[target_attr].value_counts())
print("Test set naive model total xG", sum(test_predictions[:,1]))
print("Statsbomb total xG:", sum(test_cap_x_df['shot_statsbomb_xg']))

Test set actual goal information
shot_outcome
0    46
1     3
Name: count, dtype: int64
Test set naive model total xG 2.3589363589129144
Statsbomb total xG: 3.5948024983999995


In [43]:
print(test_y_df[target_attr].value_counts())

shot_outcome
0    46
1     3
Name: count, dtype: int64

In [45]:
print("Statsbomb total xG:", sum(test_cap_x_df['shot_statsbomb_xg']))

3.5948024983999995

In [ ]:
# events = sb.events(match_id=3754078)

In [ ]:
# type(events)

In [ ]:
# events.columns

In [ ]:
relevant_attributes = ['minute', 'team', 'shot_statsbomb_xg','shot_outcome', 'location', 'shot_aerial_won', 'shot_body_part', 'shot_deflected',
       'shot_end_location', 'shot_first_time', 'shot_one_on_one', 
       'shot_saved_off_target',  'shot_technique',
       'shot_type', 'type']

In [ ]:
# relevant = events[relevant_attributes]
# events[events['type']=='Own Goal Against'].dropna(axis=1)

In [ ]:
# chelsea_swansea = events[relevant_attributes].dropna(subset=['shot_statsbomb_xg'])
# type(chelsea_swansea)

In [ ]:
# base_url = 'https://understat.com/match/'
# match = str(input('Please enter match id: '))
# url = base_url + match

In [ ]:
# res = requests.get(url)
# soup = BeautifulSoup(res.content, 'lxml')
# scripts = soup.find_all('script')
#help(BeautifulSoup)

In [ ]:
#scripts

In [ ]:
# strings = scripts[1].string

In [ ]:
#strings

In [ ]:
# ind_start = strings.index("('")+2
# ind_end = strings.index("')")

# json_data = strings[ind_start:ind_end]
# json_data = json_data.encode('utf8').decode('unicode_escape')

# data = json.loads(json_data)


In [ ]:
# data

In [ ]:
# x = []
# y = []
# minute = []
# xG = []
# h_a = []
# team = []
# result = []

# data_home = data['h']
# data_away = data['a']

# for index in range(len(data_home)):
#     for key in data_home[index]:
#         if key == 'X':
#             x.append(data_home[index][key])
#         if key == 'Y':
#             y.append(data_home[index][key])
#         if key == 'xG':
#             xG.append(data_home[index][key])
#         if key == 'h_team':
#             team.append(data_home[index][key])
#         if key == 'h_a':
#             h_a.append(data_home[index][key])
#         if key == 'minute':
#             minute.append(data_home[index][key])
#         if key == 'result':
#             result.append(data_home[index][key])

# for index in range(len(data_away)):
#     for key in data_away[index]:
#         if key == 'X':
#             x.append(data_away[index][key])
#         if key == 'Y':
#             y.append(data_away[index][key])
#         if key == 'xG':
#             xG.append(data_away[index][key])
#         if key == 'a_team':
#             team.append(data_away[index][key])
#         if key == 'h_a':
#             h_a.append(data_away[index][key])
#         if key == 'minute':
#             minute.append(data_away[index][key])
#         if key == 'result':
#             result.append(data_away[index][key])

In [ ]:
# col_names = ['minute','team', 'x', 'y', 'xG','h_a', 'result']
# df = pd.DataFrame([minute, team, x,y,xG, h_a, result], index=col_names)
# df = df.T

In [ ]:
# for col in ['x', 'y', 'xG', 'minute']:
#     df[col]=pd.to_numeric(df[col])
# df.round(decimals=5)
# df.sort_values('minute')

In [ ]:
#chelsea_swansea